# ２章　小売店のデータでデータ加工を行う１０本ノック

本章では、ある小売店の売上履歴と顧客台帳データを用いて、データ分析の素地となる「データの加工」を習得することが目的です。
実際の現場データは手入力のExcel等、決して綺麗なデータではない事が多いため、
データの揺れや整合性の担保など、汚いデータを取り扱うデータ加工を主体に進めて行きます。

### ノック１１：データを読み込んでみよう

In [1]:
import pandas as pd
uriage_data = pd.read_csv("uriage.csv")
uriage_data.head()

,purchase_date,item_name,item_price,customer_name
0,2019-06-13 18:02:34,商品A,100.0,深井菜々美
1,2019-07-13 13:05:29,商 品 S,NaN,浅田賢二
2,2019-05-11 19:42:07,商 品 a,NaN,南部慶二
3,2019-02-12 23:40:45,商品Z,2600.0,麻生莉緒
4,2019-04-22 03:09:35,商品a,NaN,平田鉄二


In [3]:
kokyaku_data = pd.read_excel("kokyaku_daicho.xlsx")
kokyaku_data.head()

,顧客名,かな,地域,メールアドレス,登録日
0,須賀ひとみ,すが ひとみ,H市,suga_hitomi@example.com,2018/01/04
1,岡田 敏也,おかだ としや,E市,okada_toshiya@example.com,42782
2,芳賀 希,はが のぞみ,A市,haga_nozomi@example.com,2018/01/07
3,荻野 愛,おぎの あい,F市,ogino_ai@example.com,42872
4,栗田 憲一,くりた けんいち,E市,kurita_kenichi@example.com,43127


### ノック１２：データの揺れを見てみよう

### ノック１３：データに揺れがあるまま集計しよう

### ノック１４：商品名の揺れを補正しよう

### ノック１５：金額欠損値の補完をしよう

### ノック１６：顧客名の揺れを補正しよう

### ノック１７：日付の揺れを補正しよう

### ノック１８：顧客名をキーに２つのデータを結合(ジョイン)しよう

### ノック１９：クレンジングしたデータをダンプしよう

### ノック２０：データを集計しよう